In [1]:
import pandas as pd
import numpy as np
domain_df = pd.read_csv("../../data/initial_data.csv")
expanded_df = pd.read_csv("../../data/expanded_data.csv")
df = pd.concat([domain_df,expanded_df])
df.drop_duplicates(inplace=True)
df.head()

,domain,domain_type
0,wisuolycossttqrj.com,dga
1,wi-wamss.org,benign
2,qcxfurnkbqidxxcl.biz,dga
3,192-168-1-1-admin.ru,benign
4,dblsiobnkjxomkmh.ru,dga


In [2]:
df['domain_type'] = pd.Categorical(df['domain_type'])

In [3]:
df['domain_type'] = df.domain_type.cat.codes
df.head()

,domain,domain_type
0,wisuolycossttqrj.com,1
1,wi-wamss.org,0
2,qcxfurnkbqidxxcl.biz,1
3,192-168-1-1-admin.ru,0
4,dblsiobnkjxomkmh.ru,1


In [4]:
import tldextract
def transform(val):
    return tldextract.extract(val).domain
df['domain']= df['domain'].apply(transform)
df.head()

,domain,domain_type
0,wisuolycossttqrj,1
1,wi-wamss,0
2,qcxfurnkbqidxxcl,1
3,192-168-1-1-admin,0
4,dblsiobnkjxomkmh,1


In [5]:
target = df.pop('domain_type')
dataset = df['domain']

In [9]:
from tensorflow.keras.preprocessing import sequence
valid_chars = {x:idx+1 for idx, x in enumerate(set(''.join(dataset)))}
max_features = len(valid_chars) + 1
maxlen = np.max([len(x) for x in dataset])

# Convert characters to int and pad
X = [[valid_chars[y] for y in x] for x in dataset]
X = sequence.pad_sequences(X, maxlen=maxlen)
y = target

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [13]:
import tensorflow as tf
trainset = tf.data.Dataset.from_tensor_slices((X_train, y_train.values)).shuffle(len(X_train)).batch(128)

In [31]:
def get_compiled_model():
    main_input = tf.keras.Input(shape=(50, ), dtype='int32', name='main_input')
    embedding = tf.keras.layers.Embedding(input_dim=128, output_dim=128,input_length=50)(main_input)
    bi_lstm = tf.keras.layers.Bidirectional(layer=tf.keras.layers.LSTM(64,return_sequences=False),merge_mode='concat')(embedding)
    output = tf.keras.layers.Dense(1, activation='sigmoid')(bi_lstm)
    model = tf.keras.Model(inputs=main_input, outputs=output)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', tf.keras.metrics.Recall()])
    return model

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0.0001, patience=1)

model = get_compiled_model()
model.fit(trainset, epochs=15, callbacks=[earlystop_callback], validation_data=testset, validation_freq=1)

Train for 434 steps, validate for 186 steps
Epoch 1/15
434/434 [==============================] - 225s 519ms/step - loss: 0.2752 - accuracy: 0.8825 - recall_2: 0.8449 - val_loss: 0.2050 - val_accuracy: 0.9176 - val_recall_2: 0.8998
Epoch 2/15
434/434 [==============================] - 219s 504ms/step - loss: 0.1963 - accuracy: 0.9206 - recall_2: 0.9068 - val_loss: 0.1820 - val_accuracy: 0.9263 - val_recall_2: 0.9170
Epoch 3/15
434/434 [==============================] - 199s 459ms/step - loss: 0.1744 - accuracy: 0.9303 - recall_2: 0.9168 - val_loss: 0.1632 - val_accuracy: 0.9369 - val_recall_2: 0.9179
Epoch 4/15
434/434 [==============================] - 200s 461ms/step - loss: 0.1535 - accuracy: 0.9373 - recall_2: 0.9229 - val_loss: 0.1450 - val_accuracy: 0.9417 - val_recall_2: 0.9405
Epoch 5/15
434/434 [==============================] - 198s 456ms/step - loss: 0.1372 - accuracy: 0.9444 - recall_2: 0.9318 - val_loss: 0.1326 - val_accuracy: 0.9464 - val_recall_2: 0.9222
Epoch 6/15
434/4

In [16]:
testset = tf.data.Dataset.from_tensor_slices((X_test, y_test.values)).batch(128)

In [17]:
model.evaluate(testset)

186/186 [==============================] - 15s 79ms/step - loss: 0.1198 - accuracy: 0.9624 - recall_1: 0.9530


[0.11977213353759819, 0.9623744, 0.9529618]

In [21]:
import pandas as pd
import numpy as np
test_df = pd.read_csv("../../data/feedback.csv")

In [22]:
test_df.columns

Index(['domain', ' submitted', ' correct'], dtype='object')

In [23]:
test_df[' correct'] = pd.Categorical(test_df[' correct'])
test_df[' correct'] = test_df[' correct'].cat.codes
test_df.head()

,domain,submitted,correct
0,cfsjdqwdhmwkiv,benign,1
1,uocoqgiusyeciouaimcauykqswsymo,benign,1
2,okdbnxoauhzrawyu,benign,1
3,m8sdetc0u81lgdcpshoxsvy,benign,1
4,qkdccn,benign,1


In [24]:
real_target = test_df.pop(' correct')
real_dataset = test_df['domain']

In [25]:
real_X = [[valid_chars[y] for y in x] for x in real_dataset]
real_X = sequence.pad_sequences(real_X, maxlen=maxlen)
real_y = real_target

In [26]:
realset = tf.data.Dataset.from_tensor_slices((real_X, real_y.values)).batch(128)

In [32]:
model.evaluate(realset)

2/2 [==============================] - 0s 171ms/step - loss: 0.5735 - accuracy: 0.7890 - recall_2: 0.7879


[0.5734701454639435, 0.78902954, 0.7878788]

In [34]:
import sklearn
probs = model.predict(realset)
sklearn.metrics.confusion_matrix(real_y.values, np.argmax(probs,axis=1))

array([[  6,   0],
       [231,   0]], dtype=int64)